In [1]:
import pandas as pd
import re
from time import time

import sys
reload(sys)
sys.setdefaultencoding('utf8')
pd.options.display.max_columns=100

In [2]:
!pwd

In [3]:
df=pd.read_csv('./bidder.pm.csv')
df.shape

(390590, 4)

In [4]:
df.head()

,pid,eid,bid,text
0,183235,342969,267948,"Hi, My name is Rakesh Nanda from India. To kno..."
1,183235,342969,1274314,hello\r\\r\n\r\\r\ninterested to be a part of ...
2,183235,342969,1297670,"Hi,\r\\r\n\r\\r\nCan you pay via GAF, as I do ..."
3,183235,342969,1394125,From the name of &quot;ALLAH&quot; the most me...
4,183235,342969,1413909,"Sir,\r\\r\nAs an Retd Sr Executive &amp; also ..."


In [5]:
test=df['text'][0]

In [6]:
print test

In [7]:
# def clean(test):
#     return re.sub(r'@\S+|#|RT|\n|http\S+','',str(test)).strip()

In [8]:
a = df['text'][29]
a

'I am very interested in the project but i have this to ask; I think payment is accepted from Paypal into Gaf account, is it?'

In [9]:
# content = df['text'].apply(clean)

In [10]:
content=df['text']
len(content)

390590

### topic modeling

In [11]:
import nltk
import string
from nltk.stem.wordnet import WordNetLemmatizer

# nltk.download('stopwords')


In [12]:
from nltk.corpus import stopwords 
stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation) 
lemmatize = WordNetLemmatizer()


In [13]:
stop = pd.read_fwf('stop.txt',names=['words'])
stopwords.update(set(stop['words'].values))
# print (list(stopwords)[:10])
stopwords.update(set(['dont','lot','wa','im','ha','doe','wa', 'able',]))

In [14]:
def cleaning(article):
    article = str(article)
    zero = "".join(i for i in article if i not in punctuation)
    
    one = " ".join([i for i in zero.lower().split() if i not in stopwords])
    
    three = " ".join(lemmatize.lemmatize(i) for i in one.split())
    return three

In [15]:
all_content=content.copy()
apple = all_content

# nltk.download('wordnet')


In [16]:
start = time()

ap_text = apple.apply(cleaning)
ap_text_list = [i.split() for i in ap_text]
print (len(ap_text_list))

('used: {:.2f}s'.format(time()-start))

UnicodeDecodeError: 'utf8' codec can't decode byte 0x92 in position 1: invalid start byte

In [ ]:
ap_text[:10]

In [ ]:
from gensim.models.phrases import Phraser, Phrases

start = time()
phs = Phrases(ap_text_list)
('used: {:.2f}s'.format(time()-start))

In [ ]:
bi_gram = Phraser(phs)
start = time()

new_bi_list = [bi_gram[i] for i in ap_text_list]

('used: {:.2f}s'.format(time()-start))

In [ ]:
start = time()

phs3=Phrases(new_bi_list)
tri_gram=Phraser(phs3)

('used: {:.2f}s'.format(time()-start))

In [ ]:
start = time()

new_tri_list = [tri_gram[i] for i in new_bi_list]

('used: {:.2f}s'.format(time()-start))

In [ ]:
len(new_tri_list)

In [ ]:
new_tri_list[0]

In [ ]:
make_dic = dict()
for i,j in zip(range(len(new_tri_list)), new_tri_list):
    make_dic[i]=j

In [ ]:
without_stop = pd.Series(make_dic)
without_stop.head()

In [ ]:
ap_text_list = list(without_stop.values)

In [ ]:
len(ap_text_list)

In [ ]:
ap_text_list[100:103]

### start building

In [ ]:
# Importing Gensim
import gensim
from gensim import corpora

from time import time
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO,
                   filename='twitter_10_8.log',filemode='w')

In [ ]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
dictionary = corpora.Dictionary(ap_text_list)
dictionary.save('twitter_10_8.dict')
(dictionary)

In [ ]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in ap_text_list]
corpora.MmCorpus.serialize('twitter_10_8.mm', doc_term_matrix)

(len(doc_term_matrix))
(doc_term_matrix[100])

In [ ]:
start = time()
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, 
#                passes=5
              )
'used: {:.2f}s'.format(time()-start)

In [ ]:
ldamodel.print_topics()

In [ ]:
ldamodel.save('twitter_10_8.model')

### load saved model , plotting

In [ ]:
from gensim.models import LdaModel
loading = LdaModel.load('twitter_10_8.model')

In [ ]:
import pyLDAvis.gensim
import gensim
from time import time
pyLDAvis.enable_notebook()

In [ ]:
d = gensim.corpora.Dictionary.load('twitter_10_8.dict')
c = gensim.corpora.MmCorpus('twitter_10_8.mm')
lda = gensim.models.LdaModel.load('twitter_10_8.model')

In [ ]:
start = time()

data = pyLDAvis.gensim.prepare(lda, c, d)

('used: {:.2f}s'.format(time()-start))

In [ ]:
pyLDAvis.save_html(data,'twitter_10_8.html')


In [ ]:
data